In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

NUM_OF_ROWS = 25000
ATTACK_NAME = 'DoS'

In [136]:
pd.set_option('display.max_columns', None)  # Show all columns

---

In [137]:
# import the attack sample dataset
dosSamples = pd.read_csv('dos_goldeneye_samples.csv')
dosSamples

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,1,40.444444,8,60,40.444444,19.178177,367.802469,536.0,52,44.666667,60,32.000000,8,21.416505,44.666667,32.000000,53.600000,0,0,0,10.009141,1.798356,10.009141,1.001001,0.588773,0.348512
1,1,0.000000,0,0,0.000000,0.000000,0.000000,0.0,0,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,0,0.054580,36.643477,0.054580,0.054580,0.054580,0.000000
2,1,398.451613,0,1448,398.451613,581.196366,337789.215401,5196.0,1448,433.000000,1428,376.631579,0,568.356375,433.000000,376.631579,0.000000,2,30,0,0.680156,45.577780,0.680156,0.138054,0.022672,0.050023
3,1,0.000000,0,0,0.000000,0.000000,0.000000,0.0,0,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,3,0.000000,0,0,0.000000,0.000000,0.000000,0.0,0,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,0,0.059988,100.019968,0.059988,0.042621,0.011998,0.015793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,1,48.447228,0,744,48.447228,136.684857,18682.750090,471876.0,744,48.447228,0,0.000000,0,0.000000,48.447228,0.000000,58.903508,2417,7323,1555,4.599534,2117.605811,4.599534,0.029508,0.000472,0.001125
331,1,166.167382,8,247,166.167382,69.746274,4864.542799,24750.0,247,155.660377,246,188.743243,8,68.689875,155.660377,188.743243,190.384615,0,0,0,4.594929,50.708074,4.594929,0.707277,0.019806,0.058261
332,1,171.913043,0,1448,171.913043,364.814177,133089.383743,2798.0,1448,279.800000,517,88.923077,0,156.012554,279.800000,88.923077,0.000000,2,22,1,0.356240,64.563210,0.356240,0.091123,0.016193,0.030332
333,0,0.000000,0,0,0.000000,0.000000,0.000000,0.0,0,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,0,1.002285,1.995440,1.002285,1.002285,1.002285,0.000000


In [138]:
dosSamples = dosSamples[dosSamples['ACK Flag Count'] > 100]

In [139]:
# print some general information about the attack samples
print(f'Dataset Shape: {dosSamples.shape}')
dosSamples.reset_index() 

Dataset Shape: (23, 26)


,index,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,12,1,60.157390,0,787,60.157390,150.569621,22671.210707,595498.0,787,60.157390,0,0.0,0,0.0,60.157390,0.0,69.608182,3089,6810,0,5.674302,1744.531719,5.674302,0.036011,0.000573,0.001272
1,16,1,60.422256,0,747,60.422256,150.808688,22743.260497,602712.0,747,60.422256,0,0.0,0,0.0,60.422256,0.0,78.365882,2753,7222,0,4.504021,2214.687755,4.504021,0.011736,0.000452,0.000831
2,26,1,59.143463,0,750,59.143463,148.634259,22092.142871,585816.0,750,59.143463,0,0.0,0,0.0,59.143463,0.0,74.512338,2762,7143,0,4.769994,2076.522623,4.769994,0.012915,0.000482,0.000908
3,29,1,48.740151,0,839,48.740151,137.349166,18864.793501,290735.0,839,48.740151,0,0.0,0,0.0,48.740151,0.0,72.865915,1794,4171,0,7.314137,815.543927,7.314137,0.958723,0.001226,0.022281
4,48,1,58.905301,0,785,58.905301,149.566209,22370.051011,566669.0,785,58.905301,0,0.0,0,0.0,58.905301,0.0,69.504354,3187,6433,0,6.273365,1533.467281,6.273365,0.017698,0.000652,0.001271
5,56,1,53.852547,0,785,53.852547,143.555384,20608.148275,482088.0,785,53.852547,0,0.0,0,0.0,53.852547,0.0,65.394466,2851,6101,0,26.310867,340.239642,26.310867,2.056007,0.002939,0.137137
6,73,1,49.453506,0,716,49.453506,137.461453,18895.651125,484496.0,716,49.453506,0,0.0,0,0.0,49.453506,0.0,60.238220,2459,7338,1316,4.417856,2217.591531,4.417856,0.017121,0.000451,0.000985
7,100,1,59.383491,0,756,59.383491,148.612387,22085.641441,584868.0,756,59.383491,0,0.0,0,0.0,59.383491,0.0,70.002154,2917,6932,0,4.968718,1982.201520,4.968718,0.104032,0.000505,0.001779
8,105,1,52.209292,0,732,52.209292,142.617872,20339.857304,358469.0,732,52.209292,0,0.0,0,0.0,52.209292,0.0,70.564764,2072,4794,0,6.747444,1017.570518,6.747444,1.017136,0.000983,0.016895
9,127,1,57.449723,0,756,57.449723,147.534256,21766.356692,550196.0,756,57.449723,0,0.0,0,0.0,57.449723,0.0,67.624877,3038,6539,0,5.909633,1620.574364,5.909633,0.030217,0.000617,0.001230


In [140]:
# find the columns that we need to synthesis data for to produce an attack dataset
columnsToGather = dosSamples.replace(0, np.nan) #replace all 0 values with null
columnsToGather = columnsToGather.dropna(how="all", axis=1).columns.tolist() #remove all columns where there are null values
columnsToGather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Size',
 'Packet Length Max',
 'Packet Length Mean',
 'Packet Length Std',
 'Packet Length Variance',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'ACK Flag Count',
 'RST Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Total',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

In [141]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
RST_FlagCount = dosSamples[dosSamples['RST Flag Count'] != 0]['RST Flag Count']

MinMaxDict = {col: (dosSamples[col].min() * 0.85, dosSamples[col].max() * 1.15) for col in columnsToGather}
MinMaxDict['Number of Ports'] = (1, 1)
MinMaxDict['RST Flag Count'] = (np.min(RST_FlagCount), np.max(RST_FlagCount))
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Size': (34.22086614173228, 69.81576753064094),
 'Packet Length Max': (608.6, 964.8499999999999),
 'Packet Length Mean': (34.22086614173228, 69.81576753064094),
 'Packet Length Std': (108.07378780203702, 175.13743208210414),
 'Packet Length Variance': (13741.110129270262, 26672.278362011875),
 'Total Length of Fwd Packet': (247124.75, 694946.1499999999),
 'Fwd Packet Length Max': (608.6, 964.8499999999999),
 'Fwd Packet Length Mean': (34.22086614173228, 69.81576753064094),
 'Fwd Segment Size Avg': (34.22086614173228, 69.81576753064094),
 'Subflow Fwd Bytes': (41.23502162690107, 90.12076452997009),
 'SYN Flag Count': (1523.2, 3829.4999999999995),
 'ACK Flag Count': (3545.35, 8681.349999999999),
 'RST Flag Count': (1316, 2790),
 'Flow Duration': (3.755177581310272, 34.76618020534515),
 'Packets Per Second': (273.853783871724, 2550.230260124876),
 'IAT Total': (3.755177581310272, 34.76618020534515),
 'IAT Max': (0.009975528717041006, 8.295302116

In [142]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Size': (34.22086614173228, 69.81576753064094),
 'Packet Length Max': (608.6, 964.8499999999999),
 'Packet Length Mean': (34.22086614173228, 69.81576753064094),
 'Packet Length Std': (108.07378780203702, 175.13743208210414),
 'Packet Length Variance': (13741.110129270262, 26672.278362011875),
 'Total Length of Fwd Packet': (247124.75, 694946.1499999999),
 'Fwd Packet Length Max': (608.6, 964.8499999999999),
 'Fwd Packet Length Mean': (34.22086614173228, 69.81576753064094),
 'Fwd Segment Size Avg': (34.22086614173228, 69.81576753064094),
 'Subflow Fwd Bytes': (41.23502162690107, 90.12076452997009),
 'SYN Flag Count': (1523, 3829),
 'ACK Flag Count': (3545, 8681),
 'RST Flag Count': (1316, 2790),
 'Flow Duration': (3.755177581310272, 34.76618020534515),
 'Packets Per Second': (273.853783871724, 2550.230260124876),
 'IAT Total': (3.755177581310272, 34.76618020534515),
 'IAT Max': (0.009975528717041006, 8.295302116870879),
 'IAT Mean': (0.0003833

### Creating the dataset

In [143]:
# creating an empty dataframe before adding values to it
dosDataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dosSamples.columns))), columns=dosSamples.columns)
dosDataset.head(3)

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
# adding zeros to all columns that should not have any values
zeroColumns = [col for col in dosSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    dosDataset[col] = int(0)
zeroColumns

['Packet Length Min',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg']

In [145]:
dosDataset.head(3)

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Calculating the correct corelation between columns

In [146]:
# inserting values that are the same across multiple columns
sameValueColumns = ['Average Packet Size', 'Packet Length Mean', 'Fwd Packet Length Mean', 'Fwd Segment Size Avg']
sameValueColumns2 = ['Packet Length Max', 'Fwd Packet Length Max']

# Generate random values for the first group of columns
randValues = np.random.uniform(MinMaxDict['Average Packet Size'][0], MinMaxDict['Average Packet Size'][1], size=NUM_OF_ROWS)

# Assign the same random value across all columns in 'sameValueColumns'
for col in sameValueColumns:
    dosDataset[col] = randValues

# Generate random values for the second group of columns
randValues2 = np.random.uniform(MinMaxDict['Packet Length Max'][0], MinMaxDict['Packet Length Max'][1], size=NUM_OF_ROWS)

# Assign the same random value across all columns in 'sameValueColumns2'
for col in sameValueColumns2:
    dosDataset[col] = randValues2

In [147]:
dosDataset

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,0.0,46.638788,0,918.408458,46.638788,0.0,0.0,0.0,918.408458,46.638788,0,0,0,0,46.638788,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,37.121723,0,926.998595,37.121723,0.0,0.0,0.0,926.998595,37.121723,0,0,0,0,37.121723,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,39.872869,0,930.452742,39.872869,0.0,0.0,0.0,930.452742,39.872869,0,0,0,0,39.872869,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,63.970784,0,920.607159,63.970784,0.0,0.0,0.0,920.607159,63.970784,0,0,0,0,63.970784,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,39.510316,0,957.415323,39.510316,0.0,0.0,0.0,957.415323,39.510316,0,0,0,0,39.510316,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,64.015910,0,714.413537,64.015910,0.0,0.0,0.0,714.413537,64.015910,0,0,0,0,64.015910,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24996,0.0,66.720253,0,781.151843,66.720253,0.0,0.0,0.0,781.151843,66.720253,0,0,0,0,66.720253,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24997,0.0,52.623784,0,717.492912,52.623784,0.0,0.0,0.0,717.492912,52.623784,0,0,0,0,52.623784,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24998,0.0,59.361434,0,953.655413,59.361434,0.0,0.0,0.0,953.655413,59.361434,0,0,0,0,59.361434,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### First group

In [148]:
# finding the correlation between the SYN Flag Count column to the rest of the columns in order to create new data
firstCorrelation = ['SYN Flag Count', 'ACK Flag Count']
independent_col = dosSamples[firstCorrelation[0]].values.reshape(-1, 1) #column 'SYN Flag Count'
dependent_cols = dosSamples[firstCorrelation[1]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(firstCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('ACK Flag Count', 2.3889997701634798)


In [149]:
dosDataset['SYN Flag Count'] = np.random.randint(MinMaxDict['SYN Flag Count'][0], MinMaxDict['SYN Flag Count'][1], NUM_OF_ROWS)

for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values
        # calculate the value we want to add into the dataset in the given row
        delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
        updatedFactor = factor + random.choice([-1, 1]) * delta
        dosDataset.loc[index, col] = int(row['SYN Flag Count'] * updatedFactor)

In [150]:
dosDataset.head(10)

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,0.0,46.638788,0,918.408458,46.638788,0.0,0.0,0.0,918.408458,46.638788,0,0,0,0,46.638788,0,0.0,2915,8102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,37.121723,0,926.998595,37.121723,0.0,0.0,0.0,926.998595,37.121723,0,0,0,0,37.121723,0,0.0,2468,6488.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,39.872869,0,930.452742,39.872869,0.0,0.0,0.0,930.452742,39.872869,0,0,0,0,39.872869,0,0.0,2306,4503.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,63.970784,0,920.607159,63.970784,0.0,0.0,0.0,920.607159,63.970784,0,0,0,0,63.970784,0,0.0,2704,5729.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,39.510316,0,957.415323,39.510316,0.0,0.0,0.0,957.415323,39.510316,0,0,0,0,39.510316,0,0.0,2821,7629.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,56.450077,0,635.293805,56.450077,0.0,0.0,0.0,635.293805,56.450077,0,0,0,0,56.450077,0,0.0,2252,5935.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,40.517837,0,663.615896,40.517837,0.0,0.0,0.0,663.615896,40.517837,0,0,0,0,40.517837,0,0.0,2492,4987.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,62.210601,0,811.247519,62.210601,0.0,0.0,0.0,811.247519,62.210601,0,0,0,0,62.210601,0,0.0,2365,4793.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,62.174802,0,855.809208,62.174802,0.0,0.0,0.0,855.809208,62.174802,0,0,0,0,62.174802,0,0.0,3051,8214.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,40.728668,0,637.263134,40.728668,0.0,0.0,0.0,637.263134,40.728668,0,0,0,0,40.728668,0,0.0,2619,5482.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [151]:
# adding the RST flag such that half of the rows will have 0 and the other half will have a random value from the known range
midpoint = len(dosDataset) // 2 #calculate the midpoint of the DataFrame

# Generate random integers for the second half
random_values = np.random.randint(MinMaxDict['RST Flag Count'][0], MinMaxDict['RST Flag Count'][1] + 1, size=len(dosDataset) - midpoint)

# Add the new column with 0s for the first half and random integers for the second half
new_column_values = [0] * midpoint + list(random_values)
dosDataset['RST Flag Count'] = new_column_values

In [152]:
dosDataset

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,0.0,46.638788,0,918.408458,46.638788,0.0,0.0,0.0,918.408458,46.638788,0,0,0,0,46.638788,0,0.0,2915,8102.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,37.121723,0,926.998595,37.121723,0.0,0.0,0.0,926.998595,37.121723,0,0,0,0,37.121723,0,0.0,2468,6488.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,39.872869,0,930.452742,39.872869,0.0,0.0,0.0,930.452742,39.872869,0,0,0,0,39.872869,0,0.0,2306,4503.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,63.970784,0,920.607159,63.970784,0.0,0.0,0.0,920.607159,63.970784,0,0,0,0,63.970784,0,0.0,2704,5729.0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,39.510316,0,957.415323,39.510316,0.0,0.0,0.0,957.415323,39.510316,0,0,0,0,39.510316,0,0.0,2821,7629.0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,64.015910,0,714.413537,64.015910,0.0,0.0,0.0,714.413537,64.015910,0,0,0,0,64.015910,0,0.0,2270,4795.0,2314,0.0,0.0,0.0,0.0,0.0,0.0
24996,0.0,66.720253,0,781.151843,66.720253,0.0,0.0,0.0,781.151843,66.720253,0,0,0,0,66.720253,0,0.0,2697,5771.0,2198,0.0,0.0,0.0,0.0,0.0,0.0
24997,0.0,52.623784,0,717.492912,52.623784,0.0,0.0,0.0,717.492912,52.623784,0,0,0,0,52.623784,0,0.0,3429,7000.0,2340,0.0,0.0,0.0,0.0,0.0,0.0
24998,0.0,59.361434,0,953.655413,59.361434,0.0,0.0,0.0,953.655413,59.361434,0,0,0,0,59.361434,0,0.0,2169,4184.0,2604,0.0,0.0,0.0,0.0,0.0,0.0


### Second group

In [153]:
# Generate random values for the first group of columns
randValues = np.random.uniform(MinMaxDict['Flow Duration'][0]*0.9, MinMaxDict['Flow Duration'][1]*1.05, size=NUM_OF_ROWS)

# Assign the same random value across all columns in `sameValueColumns`
for col in ['Flow Duration', 'IAT Total']:
    dosDataset[col] = randValues

In [154]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
secondCorrelation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dosSamples[secondCorrelation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = dosSamples[secondCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(secondCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packets Per Second', 70.77351483361475)
('IAT Max', 0.15228010421854993)
('IAT Mean', 0.0001089953172077954)
('IAT Std', 0.004918242858722099)


In [155]:
durationToPacketsCorr = [x * y for x, y in zip(dosSamples['Flow Duration'].values, dosSamples['Packets Per Second'].values)]
durationToPacketsCorr = np.mean(durationToPacketsCorr)
durationToPacketsCorr

9235.217391304348

In [156]:
#iterating over all rows we need to add values
for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: 
        # calculate a random small delta of the factor for adding some randomness
        if col == 'Packets Per Second':
            delta = random.uniform(durationToPacketsCorr * 0.1, durationToPacketsCorr * 0.2) # select a delta
            updatedFactor = durationToPacketsCorr + delta
            dosDataset.loc[index, col] = updatedFactor / row['Flow Duration']
        else:
            if col == 'IAT Std':
                delta = random.uniform(factor * 0.55, factor * 0.8)
                updatedFactor = factor + random.choices([-1, 1], weights=[2, 1], k=1)[0] * delta
            else:
                delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
                updatedFactor = factor + random.choice([-1, 1]) * delta
            dosDataset.loc[index, col] = row['Flow Duration'] * updatedFactor

In [157]:
dosDataset.describe()

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
count,25000.0,25000.000000,25000.0,25000.000000,25000.000000,25000.0,25000.0,25000.0,25000.000000,25000.000000,25000.0,25000.0,25000.0,25000.0,25000.000000,25000.0,25000.0,25000.000000,25000.00000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,0.0,51.927562,0.0,786.587857,51.927562,0.0,0.0,0.0,786.587857,51.927562,0.0,0.0,0.0,0.0,51.927562,0.0,0.0,2666.170320,6373.42204,1029.646800,19.905044,767.276381,19.905044,3.033774,0.002172,0.076388
std,0.0,10.204866,0.0,102.771767,10.204866,0.0,0.0,0.0,102.771767,10.204866,0.0,0.0,0.0,0.0,10.204866,0.0,0.0,666.980437,1890.76102,1072.317612,9.577886,581.813698,9.577886,1.548886,0.001109,0.078847
min,0.0,34.222030,0.0,608.602466,34.222030,0.0,0.0,0.0,608.602466,34.222030,0.0,0.0,0.0,0.0,34.222030,0.0,0.0,1523.000000,2914.00000,0.000000,3.380696,279.082432,3.380696,0.419548,0.000303,0.003448
25%,0.0,43.125153,0.0,697.262128,43.125153,0.0,0.0,0.0,697.262128,43.125153,0.0,0.0,0.0,0.0,43.125153,0.0,0.0,2084.750000,4864.00000,0.000000,11.572460,375.557832,11.572460,1.717479,0.001235,0.024109
50%,0.0,51.846438,0.0,786.627154,51.846438,0.0,0.0,0.0,786.627154,51.846438,0.0,0.0,0.0,0.0,51.846438,0.0,0.0,2660.000000,6199.00000,658.000000,19.940482,532.429462,19.940482,2.965984,0.002118,0.042342
75%,0.0,60.726933,0.0,875.895801,60.726933,0.0,0.0,0.0,875.895801,60.726933,0.0,0.0,0.0,0.0,60.726933,0.0,0.0,3248.000000,7608.00000,2066.000000,28.260482,918.044798,28.260482,4.207322,0.003002,0.097212
max,0.0,69.815741,0.0,964.845660,69.815741,0.0,0.0,0.0,964.845660,69.815741,0.0,0.0,0.0,0.0,69.815741,0.0,0.0,3828.000000,10928.00000,2790.000000,36.503236,3243.643142,36.503236,6.659342,0.004769,0.322270


In [158]:
x = dosDataset[dosDataset['Flow Duration']<10]
x[x['Flow Duration']>5][0:40]

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
2,0.0,39.872869,0,930.452742,39.872869,0.0,0.0,0.0,930.452742,39.872869,0,0,0,0,39.872869,0,0.0,2306,4503.0,0,9.225345,1166.478482,9.225345,1.207826,0.001203,0.009753
3,0.0,63.970784,0,920.607159,63.970784,0.0,0.0,0.0,920.607159,63.970784,0,0,0,0,63.970784,0,0.0,2704,5729.0,0,7.280933,1466.798391,7.280933,0.906136,0.000939,0.011996
6,0.0,40.517837,0,663.615896,40.517837,0.0,0.0,0.0,663.615896,40.517837,0,0,0,0,40.517837,0,0.0,2492,4987.0,0,6.660029,1553.618036,6.660029,1.201053,0.000871,0.008040
13,0.0,64.450279,0,776.105886,64.450279,0.0,0.0,0.0,776.105886,64.450279,0,0,0,0,64.450279,0,0.0,2658,7300.0,0,8.560479,1286.602547,8.560479,1.451970,0.000798,0.013856
26,0.0,66.882549,0,815.800237,66.882549,0.0,0.0,0.0,815.800237,66.882549,0,0,0,0,66.882549,0,0.0,2125,6083.0,0,5.195036,2051.427422,5.195036,0.884997,0.000488,0.011339
29,0.0,58.547951,0,850.898956,58.547951,0.0,0.0,0.0,850.898956,58.547951,0,0,0,0,58.547951,0,0.0,3027,8271.0,0,7.136930,1546.971847,7.136930,0.965788,0.000876,0.012474
31,0.0,60.459759,0,895.941465,60.459759,0.0,0.0,0.0,895.941465,60.459759,0,0,0,0,60.459759,0,0.0,1814,5105.0,0,6.757235,1622.828407,6.757235,1.223747,0.000881,0.008289
50,0.0,48.646911,0,808.998390,48.646911,0.0,0.0,0.0,808.998390,48.646911,0,0,0,0,48.646911,0,0.0,2569,5460.0,0,8.707567,1200.894819,8.707567,1.574069,0.001101,0.072352
54,0.0,41.731671,0,777.196701,41.731671,0.0,0.0,0.0,777.196701,41.731671,0,0,0,0,41.731671,0,0.0,1765,3613.0,0,6.145910,1802.639186,6.145910,0.822972,0.000598,0.007160
58,0.0,42.288139,0,732.585611,42.288139,0.0,0.0,0.0,732.585611,42.288139,0,0,0,0,42.288139,0,0.0,2396,4989.0,0,9.213190,1151.691610,9.213190,1.152807,0.000833,0.010372


In [159]:
x = dosDataset[dosDataset['Flow Duration']<50.5]
x[x['Flow Duration']>25][20:40]

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
48,0.0,42.949479,0,701.069350,42.949479,0.0,0.0,0.0,701.069350,42.949479,0,0,0,0,42.949479,0,0.0,2985,7969.0,0,28.658601,367.844973,28.658601,3.864317,0.003556,0.249917
60,0.0,51.296884,0,773.275489,51.296884,0.0,0.0,0.0,773.275489,51.296884,0,0,0,0,51.296884,0,0.0,3422,9123.0,0,35.554366,305.052635,35.554366,6.471089,0.003142,0.280246
63,0.0,55.900436,0,735.803198,55.900436,0.0,0.0,0.0,735.803198,55.900436,0,0,0,0,55.900436,0,0.0,2392,5139.0,0,31.267497,339.028587,31.267497,5.322888,0.003014,0.051766
64,0.0,35.718453,0,838.897591,35.718453,0.0,0.0,0.0,838.897591,35.718453,0,0,0,0,35.718453,0,0.0,1916,5079.0,0,27.052636,381.820417,27.052636,3.475127,0.002614,0.038555
67,0.0,58.547068,0,924.263594,58.547068,0.0,0.0,0.0,924.263594,58.547068,0,0,0,0,58.547068,0,0.0,3274,8799.0,0,32.359506,316.543388,32.359506,4.330501,0.002877,0.059589
68,0.0,40.274604,0,784.785811,40.274604,0.0,0.0,0.0,784.785811,40.274604,0,0,0,0,40.274604,0,0.0,1700,4632.0,0,27.293273,391.715818,27.293273,3.718066,0.003439,0.043828
69,0.0,48.941737,0,743.509260,48.941737,0.0,0.0,0.0,743.509260,48.941737,0,0,0,0,48.941737,0,0.0,2166,5975.0,0,34.859996,302.579277,34.859996,6.250375,0.003234,0.286862
71,0.0,55.931436,0,653.665222,55.931436,0.0,0.0,0.0,653.665222,55.931436,0,0,0,0,55.931436,0,0.0,2436,6672.0,0,32.094673,332.790560,32.094673,5.491982,0.003127,0.260833
76,0.0,45.056088,0,658.673744,45.056088,0.0,0.0,0.0,658.673744,45.056088,0,0,0,0,45.056088,0,0.0,2486,4823.0,0,27.796613,371.579641,27.796613,3.766447,0.002687,0.043928
78,0.0,56.324592,0,762.845997,56.324592,0.0,0.0,0.0,762.845997,56.324592,0,0,0,0,56.324592,0,0.0,2309,6212.0,0,31.678822,322.711346,31.678822,4.215873,0.003055,0.044513


In [160]:
dosDataset[dosDataset['Flow Duration']<6][:20]

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
19,0.0,60.496349,0,726.448349,60.496349,0.0,0.0,0.0,726.448349,60.496349,0,0,0,0,60.496349,0,0.0,3530,7040.0,0,4.780631,2230.399727,4.780631,0.856376,0.000599,0.007920
26,0.0,66.882549,0,815.800237,66.882549,0.0,0.0,0.0,815.800237,66.882549,0,0,0,0,66.882549,0,0.0,2125,6083.0,0,5.195036,2051.427422,5.195036,0.884997,0.000488,0.011339
39,0.0,59.184989,0,663.216988,59.184989,0.0,0.0,0.0,663.216988,59.184989,0,0,0,0,59.184989,0,0.0,1731,4847.0,0,3.886895,2795.732354,3.886895,0.521287,0.000367,0.008356
65,0.0,64.079184,0,802.165092,64.079184,0.0,0.0,0.0,802.165092,64.079184,0,0,0,0,64.079184,0,0.0,3669,10508.0,0,4.177971,2582.601061,4.177971,0.532134,0.000509,0.032820
66,0.0,35.115083,0,882.505474,35.115083,0.0,0.0,0.0,882.505474,35.115083,0,0,0,0,35.115083,0,0.0,3351,9082.0,0,4.711865,2198.013405,4.711865,0.792830,0.000604,0.007954
74,0.0,59.436152,0,867.228897,59.436152,0.0,0.0,0.0,867.228897,59.436152,0,0,0,0,59.436152,0,0.0,3273,6454.0,0,5.629484,1917.977970,5.629484,0.697987,0.000538,0.011517
77,0.0,65.676986,0,683.093198,65.676986,0.0,0.0,0.0,683.093198,65.676986,0,0,0,0,65.676986,0,0.0,2239,5899.0,0,3.412597,3018.095402,3.412597,0.425749,0.000305,0.006572
82,0.0,68.645741,0,710.079511,68.645741,0.0,0.0,0.0,710.079511,68.645741,0,0,0,0,68.645741,0,0.0,1955,5499.0,0,3.771173,2859.494416,3.771173,0.490763,0.000366,0.007598
97,0.0,54.231474,0,758.933727,54.231474,0.0,0.0,0.0,758.933727,54.231474,0,0,0,0,54.231474,0,0.0,2782,5367.0,0,5.038096,2094.661904,5.038096,0.651907,0.000607,0.010210
163,0.0,39.508855,0,825.909135,39.508855,0.0,0.0,0.0,825.909135,39.508855,0,0,0,0,39.508855,0,0.0,3235,8570.0,0,4.795792,2191.498904,4.795792,0.821193,0.000445,0.039711


### Third group

In [161]:
# finding the correlation between the Packet Length Std column to the rest of the columns in order to create new data
firstCorrelation = ['Packet Length Std', 'Packet Length Variance', 'Total Length of Fwd Packet', 'Subflow Fwd Bytes']
independent_col = dosSamples[firstCorrelation[0]].values.reshape(-1, 1) #column 'Packet Length Std'
dependent_cols = dosSamples[firstCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(firstCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packet Length Variance', 144.86044004300214)
('Total Length of Fwd Packet', 3537.389626866308)
('Subflow Fwd Bytes', 0.45655464213743985)


In [162]:
dosDataset['Packet Length Std'] = np.random.uniform(MinMaxDict['Packet Length Std'][0], MinMaxDict['Packet Length Std'][1], NUM_OF_ROWS)

for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values
        # calculate the value we want to add into the dataset in the given row
        delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
        updatedFactor = factor + random.choice([-1, 1]) * delta
        dosDataset.loc[index, col] = row['Packet Length Std'] * updatedFactor

In [163]:
x = dosDataset[dosDataset['Packet Length Std'] > 114]
x[x['Packet Length Std'] < 120].tail(20)

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
24751,0.0,56.510049,0,928.361815,56.510049,114.511648,13973.563074,456166.648541,928.361815,56.510049,0,0,0,0,56.510049,0,44.026562,2116,5653.0,1803,32.979589,328.342527,32.979589,5.723271,0.002941,0.284408
24757,0.0,54.116160,0,687.274450,54.116160,116.869735,14577.465795,459069.427943,687.274450,54.116160,0,0,0,0,54.116160,0,43.222135,2534,7003.0,2051,21.432146,509.919311,21.432146,2.837613,0.002048,0.187285
24759,0.0,59.958378,0,716.932133,59.958378,117.944591,19555.730891,482949.568468,716.932133,59.958378,0,0,0,0,59.958378,0,63.306075,2603,6848.0,2714,30.351355,338.956466,30.351355,4.126423,0.002866,0.032175
24772,0.0,51.640066,0,656.340483,51.640066,114.081324,19518.749945,444843.183142,656.340483,51.640066,0,0,0,0,51.640066,0,45.586202,1580,3230.0,1316,27.639376,385.512720,27.639376,4.896585,0.003456,0.222833
24776,0.0,57.102093,0,733.079403,57.102093,114.706518,14328.019387,337880.591738,733.079403,57.102093,0,0,0,0,57.102093,0,45.213632,2013,4076.0,2678,9.769046,1043.579134,9.769046,1.259509,0.000875,0.012976
24778,0.0,38.723261,0,653.489132,38.723261,119.277924,14900.494276,495979.225557,653.489132,38.723261,0,0,0,0,38.723261,0,48.325721,3038,8519.0,1402,21.844975,497.153426,21.844975,3.929172,0.002066,0.184362
24788,0.0,50.859166,0,692.423783,50.859166,119.508844,13986.151121,343187.644418,692.423783,50.859166,0,0,0,0,50.859166,0,44.266736,1637,3395.0,2614,24.373853,448.823412,24.373853,3.330896,0.002987,0.032647
24803,0.0,43.868108,0,874.620025,43.868108,114.421625,13483.811407,341083.851970,874.620025,43.868108,0,0,0,0,43.868108,0,61.779672,2996,8548.0,1466,26.692724,396.610362,26.692724,3.430444,0.002434,0.039618
24805,0.0,42.089938,0,726.137747,42.089938,119.700116,20356.824519,342743.883213,726.137747,42.089938,0,0,0,0,42.089938,0,64.113706,2406,6701.0,1969,13.937819,748.405158,13.937819,2.376395,0.001735,0.021831
24825,0.0,62.323438,0,897.592031,62.323438,115.705849,18886.675313,327909.209459,897.592031,62.323438,0,0,0,0,62.323438,0,44.492645,1545,3314.0,1357,12.174525,846.036523,12.174525,1.532252,0.001549,0.016256


### Adding labels and verifiying the dataset

In [164]:
# adding number of ports and a label to the dataset
dosDataset['Number of Ports'] = np.full(shape=NUM_OF_ROWS, fill_value=1, dtype=int)
dosDataset['Label'] = ATTACK_NAME

In [165]:
dosDataset

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std,Label
0,1,46.638788,0,918.408458,46.638788,167.008349,27560.657596,665505.173977,918.408458,46.638788,0,0,0,0,46.638788,0,84.395959,2915,8102.0,0,35.538429,294.713699,35.538429,6.396122,0.004318,0.310365,DoS
1,1,37.121723,0,926.998595,37.121723,111.650394,18652.157674,324569.720136,926.998595,37.121723,0,0,0,0,37.121723,0,59.027068,2468,6488.0,0,12.825850,803.767248,12.825850,2.213862,0.001213,0.100870,DoS
2,1,39.872869,0,930.452742,39.872869,114.453625,13294.169499,484804.800638,930.452742,39.872869,0,0,0,0,39.872869,0,42.379073,2306,4503.0,0,9.225345,1166.478482,9.225345,1.207826,0.001203,0.009753,DoS
3,1,63.970784,0,920.607159,63.970784,151.768491,18114.147919,436672.097548,920.607159,63.970784,0,0,0,0,63.970784,0,60.144434,2704,5729.0,0,7.280933,1466.798391,7.280933,0.906136,0.000939,0.011996,DoS
4,1,39.510316,0,957.415323,39.510316,147.692709,18283.056814,457599.126967,957.415323,39.510316,0,0,0,0,39.510316,0,80.731541,2821,7629.0,0,26.111718,402.946885,26.111718,3.362270,0.003200,0.045445,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1,64.015910,0,714.413537,64.015910,121.396527,14315.963164,504764.486058,714.413537,64.015910,0,0,0,0,64.015910,0,63.676608,2270,4795.0,2314,35.277584,294.138513,35.277584,4.777247,0.003269,0.275770,DoS
24996,1,66.720253,0,781.151843,66.720253,153.567832,18990.785107,614396.466645,781.151843,66.720253,0,0,0,0,66.720253,0,80.791118,2697,5771.0,2198,15.041129,715.914073,15.041129,2.607898,0.001443,0.030212,DoS
24997,1,52.623784,0,717.492912,52.623784,126.084260,15750.818348,520369.281829,717.492912,52.623784,0,0,0,0,52.623784,0,64.412811,3429,7000.0,2340,8.812453,1244.493870,8.812453,1.587776,0.001109,0.075483,DoS
24998,1,59.361434,0,953.655413,59.361434,148.319277,23930.655276,441408.342708,953.655413,59.361434,0,0,0,0,59.361434,0,79.028471,2169,4184.0,2604,23.924752,426.926434,23.924752,4.139136,0.002291,0.207306,DoS


In [166]:
dosSamples.describe()

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
count,23.0,23.000000,23.0,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.0,23.0,23.0,23.0,23.000000,23.0,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000
mean,1.0,54.673019,0.0,762.565217,54.673019,144.196338,20841.805224,507546.391304,762.565217,54.673019,0.0,0.0,0.0,0.0,54.673019,0.0,65.701451,2675.869565,6529.434783,423.782609,8.903293,1473.342044,8.903293,1.022834,0.000985,0.027354
std,0.0,6.185297,0.0,27.197637,6.185297,7.173471,2017.847548,95777.141243,27.197637,6.185297,0.0,0.0,0.0,0.0,6.185297,0.0,7.864561,460.455042,871.619382,870.883770,7.317741,669.600578,7.317741,1.885191,0.000794,0.059814
min,1.0,40.259843,0.0,716.000000,40.259843,127.145633,16166.011917,290735.000000,716.000000,40.259843,0.0,0.0,0.0,0.0,40.259843,0.0,48.511790,1792.000000,4171.000000,0.000000,4.417856,322.180922,4.417856,0.011736,0.000451,0.000831
25%,1.0,49.387123,0.0,745.500000,49.387123,138.889110,19290.249649,476130.000000,745.500000,49.387123,0.0,0.0,0.0,0.0,49.387123,0.0,59.345865,2431.500000,6404.500000,0.000000,4.655518,890.599042,4.655518,0.026170,0.000492,0.001078
50%,1.0,57.719220,0.0,756.000000,57.719220,147.534256,21766.356692,550196.000000,756.000000,57.719220,0.0,0.0,0.0,0.0,57.719220,0.0,67.624877,2762.000000,6711.000000,0.000000,5.909633,1627.093982,5.909633,0.036011,0.000615,0.001331
75%,1.0,59.684044,0.0,784.000000,59.684044,149.613596,22384.230401,582170.500000,784.000000,59.684044,0.0,0.0,0.0,0.0,59.684044,0.0,71.635163,3063.500000,7182.500000,0.000000,9.100059,2035.161840,9.100059,0.987929,0.001123,0.019328
max,1.0,60.709363,0.0,839.000000,60.709363,152.293419,23193.285532,604301.000000,839.000000,60.709363,0.0,0.0,0.0,0.0,60.709363,0.0,78.365882,3330.000000,7549.000000,2790.000000,30.231461,2217.591531,30.231461,7.213306,0.003104,0.251709


In [167]:
dosDataset.describe()

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
count,25000.0,25000.000000,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.0,25000.0,25000.0,25000.0,25000.000000,25000.0,25000.000000,25000.000000,25000.00000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,1.0,51.927562,0.0,786.587857,51.927562,141.631110,20509.910143,501623.557975,786.587857,51.927562,0.0,0.0,0.0,0.0,51.927562,0.0,64.626339,2666.170320,6373.42204,1029.646800,19.905044,767.276381,19.905044,3.033774,0.002172,0.076388
std,0.0,10.204866,0.0,102.771767,10.204866,19.411674,4222.007934,102713.905008,102.771767,10.204866,0.0,0.0,0.0,0.0,10.204866,0.0,13.328293,666.980437,1890.76102,1072.317612,9.577886,581.813698,9.577886,1.548886,0.001109,0.078847
min,1.0,34.222030,0.0,608.602466,34.222030,108.075180,12590.743309,306500.725463,608.602466,34.222030,0.0,0.0,0.0,0.0,34.222030,0.0,39.645050,1523.000000,2914.00000,0.000000,3.380696,279.082432,3.380696,0.419548,0.000303,0.003448
25%,1.0,43.125153,0.0,697.262128,43.125153,124.853789,17420.933263,427038.052012,697.262128,43.125153,0.0,0.0,0.0,0.0,43.125153,0.0,54.902220,2084.750000,4864.00000,0.000000,11.572460,375.557832,11.572460,1.717479,0.001235,0.024109
50%,1.0,51.846438,0.0,786.627154,51.846438,141.559752,20030.165079,490120.430720,786.627154,51.846438,0.0,0.0,0.0,0.0,51.846438,0.0,63.110165,2660.000000,6199.00000,658.000000,19.940482,532.429462,19.940482,2.965984,0.002118,0.042342
75%,1.0,60.726933,0.0,875.895801,60.726933,158.591351,23564.097491,576110.119575,875.895801,60.726933,0.0,0.0,0.0,0.0,60.726933,0.0,74.185219,3248.000000,7608.00000,2066.000000,28.260482,918.044798,28.260482,4.207322,0.003002,0.097212
max,1.0,69.815741,0.0,964.845660,69.815741,175.136368,30391.329736,743089.379144,964.845660,69.815741,0.0,0.0,0.0,0.0,69.815741,0.0,95.667265,3828.000000,10928.00000,2790.000000,36.503236,3243.643142,36.503236,6.659342,0.004769,0.322270


In [168]:
# save the dataset
# dosDataset.to_csv('dos_goldeneye_dataset_updated_flows.csv', index=False)

---